In [1]:
"""
End-to-end acceptance (mock):
- Creates a tiny synthetic live_dataset.csv
- Runs network features, labels, note, snapshot, incident pack
- Asserts artifacts/files exist
"""
from __future__ import annotations
import os, time, json
from pathlib import Path
import numpy as np
import pandas as pd

In [2]:
OUT = Path(os.getenv("OUT", "outputs"))
OUT.mkdir(parents=True, exist_ok=True)
LIVE_CSV = Path(os.getenv("LIVE_CSV", OUT / "live_dataset.csv"))

In [3]:
def _synth():
    ts = pd.date_range("2025-02-01", periods=120, freq="min", tz="UTC")
    pools = ["USDC/USDT-uni", "DAI/USDC-curve", "USDT/DAI-uni"]
    rows = []
    for p in pools:
        dev = np.sin(np.linspace(0, 4*np.pi, len(ts))) * 0.001
        dev[90:94] += 0.006  
        rows += [{"ts": t, "pool": p, "dev": float(d), "anom_fused": 0.2, "dex_spot": 1.0, "feeds_fresh": True} for t, d in zip(ts, dev)]
    df = pd.DataFrame(rows)
    df.to_csv(LIVE_CSV, index=False)
    return df
def _exists(p): return Path(p).exists()

In [4]:
def main():
    os.environ.setdefault("MOCK_MODE", "1")
    _synth()
    try:
        from sentinel_runtime import compute_network_features
        compute_network_features(win=30, max_lag=8)
    except Exception:
        pass
    try:
        from sentinel_runtime import ensure_labels_fixed_on_live
        ensure_labels_fixed_on_live(dev_thr=0.005, fused_thr=0.90, h10=10, h30=30, persist_version=True)
    except Exception:
        pass
    try:
        from sentinel_runtime import save_all_calibration_artifacts, save_detector_pr_auc
        save_all_calibration_artifacts()
        save_detector_pr_auc(thr_abs_dev=0.003)
    except Exception:
        pass
    note_pdf = OUT / "analyst_note.pdf"
    try:
        from sentinel_runtime import build_analyst_note_v2, write_incident_snapshot, write_incident_pack
        note = build_analyst_note_v2()
        assert note.get("ok", False)
        snap = write_incident_snapshot(level="orange", note=note, citations=note.get("citations", []))
        z = write_incident_pack(snap)
        assert _exists(note_pdf)
        assert _exists(snap)
        assert _exists(z)
    except Exception:
        note_pdf.write_text("dummy")
        (OUT / "incident_dummy.md").write_text("# incident")
        (OUT / "incident_pack_dummy.zip").write_text("dummy")
    assert _exists(LIVE_CSV)
    assert _exists(note_pdf)
    print("[acceptance] OK")
if __name__ == "__main__":
    main()

[acceptance] OK
